Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
# from __future__ import print_function # Using Python 3 interpreter
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 1 to [0.0, 1.0, 0.0 ...], 2 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [4]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

Regularization looks like this: ![Regularization formula](https://raw.githubusercontent.com/ritchieng/machine-learning-nanodegree/master/deep_learning/deep_neural_nets/dnn12.png)

We will need to establish a beta... Note: other conventions use a lambda instead

*Image Credit: Ritchie Ng*

**Multinomial logistic regression with SGD and L2 regularized loss function**

In [5]:
beta = 0.01 # A good beta to start with, tuning beta can give better performance

In [6]:
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  # original loss function
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits))
  # Loss function using L2 Regularization
  regularization_term = tf.nn.l2_loss(weights)
  loss = tf.reduce_mean(loss + beta * regularization_term)# update loss fxn
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Run Computation

In [7]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 48.455978
Minibatch accuracy: 10.9%
Validation accuracy: 10.0%
Minibatch loss at step 500: 0.885267
Minibatch accuracy: 81.2%
Validation accuracy: 80.3%
Minibatch loss at step 1000: 1.007396
Minibatch accuracy: 68.8%
Validation accuracy: 80.7%
Minibatch loss at step 1500: 0.943685
Minibatch accuracy: 75.0%
Validation accuracy: 81.6%
Minibatch loss at step 2000: 0.835148
Minibatch accuracy: 78.9%
Validation accuracy: 81.2%
Minibatch loss at step 2500: 0.499671
Minibatch accuracy: 87.5%
Validation accuracy: 81.7%
Minibatch loss at step 3000: 0.722447
Minibatch accuracy: 82.8%
Validation accuracy: 81.2%
Test accuracy: 88.1%


**Neural Network with L2 Regularization**

One hidden layer of ReLU's

In [8]:
num_nodes = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes]))
  biases1 = tf.Variable(tf.zeros([num_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu_layer = tf.nn.relu(logits1)
  logits2 = tf.matmul(relu_layer, weights2) + biases2
  # original loss function
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits2))# the last logits is the culmination
  # Add regularization terms for each weight tensor and add to original loss
  regularization_terms = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
  loss = tf.reduce_mean(loss + beta * regularization_terms)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)# the last logits is the culmination
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),weights2)+biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),weights2)+biases2)

Run computation

In [9]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3401.092285
Minibatch accuracy: 14.8%
Validation accuracy: 30.9%
Minibatch loss at step 500: 21.276815
Minibatch accuracy: 82.8%
Validation accuracy: 84.1%
Minibatch loss at step 1000: 1.138921
Minibatch accuracy: 71.9%
Validation accuracy: 83.2%
Minibatch loss at step 1500: 0.851985
Minibatch accuracy: 76.6%
Validation accuracy: 83.2%
Minibatch loss at step 2000: 0.792618
Minibatch accuracy: 83.6%
Validation accuracy: 83.6%
Minibatch loss at step 2500: 0.512264
Minibatch accuracy: 93.0%
Validation accuracy: 83.7%
Minibatch loss at step 3000: 0.690643
Minibatch accuracy: 84.4%
Validation accuracy: 83.3%
Test accuracy: 89.6%


---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

We will use a subset of the train_dataset while maintaining batch size to give us fewer batches overall

In [10]:
num_steps = 3001

# Subset of train_dataset:
train_dataset_sub = train_dataset[:500, :]
train_labels_sub = train_labels[:500]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_sub.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_sub[offset:(offset + batch_size), :]
    batch_labels = train_labels_sub[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3442.041992
Minibatch accuracy: 11.7%
Validation accuracy: 36.5%
Minibatch loss at step 500: 21.105276
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 1000: 0.476099
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Minibatch loss at step 1500: 0.303138
Minibatch accuracy: 100.0%
Validation accuracy: 77.6%
Minibatch loss at step 2000: 0.285719
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 2500: 0.277137
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 3000: 0.272303
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Test accuracy: 84.9%


Working with fewer batches results in higher (100%) minibatch accuracy, but very low validation set accuracy

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

In [11]:
num_nodes = 1024
batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes]))
  biases1 = tf.Variable(tf.zeros([num_nodes]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_nodes, num_labels]))
  biases2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu_layer = tf.nn.relu(logits1)
  keep_prob = tf.placeholder("float") # probability of node being kept in layer
  relu_layer_dropout = tf.nn.dropout(relu_layer, keep_prob=keep_prob)
    
  logits2 = tf.matmul(relu_layer_dropout, weights2) + biases2 #pass dropout layer on
  # original loss function
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(labels=tf_train_labels, logits=logits2))# the last logits is the culmination
  # Add regularization terms for each weight tensor and add to original loss
  regularization_terms = tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2)
  loss = tf.reduce_mean(loss + beta * regularization_terms)
    
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits2)# the last logits is the culmination
  
  # Avoid applying any dropout in prediction tensors for validation or testing  
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights1) + biases1),weights2)+biases2)
  test_prediction = tf.nn.softmax(tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights1) + biases1),weights2)+biases2)

Run computation

In [12]:
num_steps = 3001

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {
        tf_train_dataset : batch_data,
        tf_train_labels : batch_labels,
        keep_prob : 0.5
    }
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3622.974365
Minibatch accuracy: 17.2%
Validation accuracy: 32.5%
Minibatch loss at step 500: 21.371738
Minibatch accuracy: 82.0%
Validation accuracy: 83.8%
Minibatch loss at step 1000: 1.245984
Minibatch accuracy: 69.5%
Validation accuracy: 82.7%
Minibatch loss at step 1500: 0.924596
Minibatch accuracy: 79.7%
Validation accuracy: 83.5%
Minibatch loss at step 2000: 0.865245
Minibatch accuracy: 78.9%
Validation accuracy: 83.3%
Minibatch loss at step 2500: 0.618922
Minibatch accuracy: 87.5%
Validation accuracy: 83.2%
Minibatch loss at step 3000: 0.728835
Minibatch accuracy: 84.4%
Validation accuracy: 83.1%
Test accuracy: 89.5%


89.5 is just slightly less performance than L2 regularixation gave us

With **Extreme Overfitting**...

In [13]:
num_steps = 3001

# Subset of train_dataset:
train_dataset_sub = train_dataset[:500, :]
train_labels_sub = train_labels[:500]

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels_sub.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_sub[offset:(offset + batch_size), :]
    batch_labels = train_labels_sub[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {
        tf_train_dataset : batch_data,
        tf_train_labels : batch_labels,
        keep_prob : 0.5
    }
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 3629.324707
Minibatch accuracy: 14.1%
Validation accuracy: 34.9%
Minibatch loss at step 500: 21.152437
Minibatch accuracy: 99.2%
Validation accuracy: 78.4%
Minibatch loss at step 1000: 0.492334
Minibatch accuracy: 100.0%
Validation accuracy: 78.2%
Minibatch loss at step 1500: 0.334853
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 2000: 0.317805
Minibatch accuracy: 100.0%
Validation accuracy: 77.5%
Minibatch loss at step 2500: 0.292700
Minibatch accuracy: 100.0%
Validation accuracy: 77.8%
Minibatch loss at step 3000: 0.293014
Minibatch accuracy: 100.0%
Validation accuracy: 78.1%
Test accuracy: 85.3%


We have an ever-so-slightly-better performance

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [18]:
#4 hidden layers
num_nodes_layer_1 = 1024
num_nodes_layer_2 = 512
num_nodes_layer_3 = 256
num_nodes_layer_4 = 128

batch_size = 128

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Weight variables for 4 hidden layers.
  weights1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_nodes_layer_1], stddev=np.sqrt(2.0 / (image_size * image_size))))
  biases1 = tf.Variable(tf.zeros([num_nodes_layer_1]))
  weights2 = tf.Variable(
    tf.truncated_normal([num_nodes_layer_1, num_nodes_layer_2], stddev=np.sqrt(2.0 / (num_nodes_layer_1))))
  biases2 = tf.Variable(tf.zeros([num_nodes_layer_2]))
  weights3 = tf.Variable(
    tf.truncated_normal([num_nodes_layer_2, num_nodes_layer_3], stddev=np.sqrt(2.0 / (num_nodes_layer_2))))
  biases3 = tf.Variable(tf.zeros([num_nodes_layer_3]))
  weights4 = tf.Variable(
    tf.truncated_normal([num_nodes_layer_3, num_nodes_layer_4], stddev=np.sqrt(2.0 / (num_nodes_layer_3))))
  biases4 = tf.Variable(tf.zeros([num_nodes_layer_4]))
  weights5 = tf.Variable(# output layer
    tf.truncated_normal([num_nodes_layer_4, num_labels], stddev=np.sqrt(2.0 / (num_nodes_layer_4))))
  biases5 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  keep_prob = tf.placeholder(tf.float32) # probability of node being kept in layer

  logits1 = tf.matmul(tf_train_dataset, weights1) + biases1
  relu_layer_dropout_1 = tf.nn.dropout(tf.nn.relu(logits1), keep_prob)
  logits2 = tf.matmul(relu_layer_dropout_1, weights2) + biases2
  relu_layer_dropout_2 = tf.nn.dropout(tf.nn.relu(logits2), keep_prob)
  logits3 = tf.matmul(relu_layer_dropout_2, weights3) + biases3
  relu_layer_dropout_3 = tf.nn.dropout(tf.nn.relu(logits3), keep_prob)
  logits4 = tf.matmul(relu_layer_dropout_3, weights4) + biases4
  relu_layer_dropout_4 = tf.nn.dropout(tf.nn.relu(logits4), keep_prob)
  logits5 = tf.matmul(relu_layer_dropout_4, weights5) + biases5 # output layer

  # original loss function
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits5, labels=tf_train_labels))# the last logits tensor is the culmination
  # Add regularization terms for each weight tensor and add to original loss
  regularization_terms =\
    tf.nn.l2_loss(weights1) + tf.nn.l2_loss(weights2) + tf.nn.l2_loss(weights3) +\
    tf.nn.l2_loss(weights4) + tf.nn.l2_loss(weights5)
  temp_beta = 0.001438
  loss = tf.reduce_mean(loss + temp_beta * regularization_terms)
    
  # Optimizer.
  global_step = tf.Variable(0)  # count the number of steps taken.
  learning_rate = tf.train.exponential_decay(0.1, global_step, 100000, 0.96, staircase=True)
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits5)# the last logits is the culmination
  
  # Avoid applying any dropout in prediction tensors for validation or testing  
  # VALIDATION
  valid_logits_1 = tf.matmul(tf_valid_dataset, weights1) + biases1
  relu_layer_1 = tf.nn.relu(valid_logits_1)
  valid_logits_2 = tf.matmul(relu_layer_1, weights2) + biases2
  relu_layer_2 = tf.nn.relu(valid_logits_2)
  valid_logits_3 = tf.matmul(relu_layer_2, weights3) + biases3
  relu_layer_3 = tf.nn.relu(valid_logits_3)
  valid_logits_4 = tf.matmul(relu_layer_3, weights4) + biases4
  relu_layer_4 = tf.nn.relu(valid_logits_4)
  valid_logits_5 = tf.matmul(relu_layer_4, weights5) + biases5 # output layer
  valid_prediction = tf.nn.softmax(valid_logits_5)

  # TESTING
  test_logits_1 = tf.matmul(tf_test_dataset, weights1) + biases1
  relu_layer_1 = tf.nn.relu(test_logits_1)
  test_logits_2 = tf.matmul(relu_layer_1, weights2) + biases2
  relu_layer_2 = tf.nn.relu(test_logits_2)
  test_logits_3 = tf.matmul(relu_layer_2, weights3) + biases3
  relu_layer_3 = tf.nn.relu(test_logits_3)
  test_logits_4 = tf.matmul(relu_layer_3, weights4) + biases4
  relu_layer_4 = tf.nn.relu(test_logits_4)
  test_logits_5 = tf.matmul(relu_layer_4, weights5) + biases5 # output layer
  test_prediction = tf.nn.softmax(test_logits_5)
  

In [19]:
num_steps = 15000

with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {
        tf_train_dataset : batch_data,
        tf_train_labels : batch_labels,
        keep_prob : 0.7
    }
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict
    )
    if (step % 500 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 4.634124
Minibatch accuracy: 10.9%
Validation accuracy: 19.2%
Minibatch loss at step 500: 2.366647
Minibatch accuracy: 83.6%
Validation accuracy: 84.0%
Minibatch loss at step 1000: 2.401542
Minibatch accuracy: 74.2%
Validation accuracy: 85.1%
Minibatch loss at step 1500: 2.093631
Minibatch accuracy: 78.1%
Validation accuracy: 85.9%
Minibatch loss at step 2000: 1.752513
Minibatch accuracy: 85.2%
Validation accuracy: 86.6%
Minibatch loss at step 2500: 1.371040
Minibatch accuracy: 91.4%
Validation accuracy: 87.1%
Minibatch loss at step 3000: 1.341519
Minibatch accuracy: 89.1%
Validation accuracy: 87.2%
Minibatch loss at step 3500: 1.348024
Minibatch accuracy: 86.7%
Validation accuracy: 87.7%
Minibatch loss at step 4000: 1.125007
Minibatch accuracy: 86.7%
Validation accuracy: 87.8%
Minibatch loss at step 4500: 1.048298
Minibatch accuracy: 88.3%
Validation accuracy: 88.1%
Minibatch loss at step 5000: 1.006805
Minibatch accuracy: 89.8%
Validation accurac